In [37]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import requests
from IPython.display import display

In [38]:

def get_drug_name(classId, relaSource):
    '''Return the correct url for rxcui'''
    url_base = 'https://rxnav.nlm.nih.gov/REST/rxclass/classMembers.json?classId='
    my_url = 'https://rxnav.nlm.nih.gov/REST/rxclass/classMembers.json?classId=' + classId + '&relaSource=' + relaSource

    r = requests.get(url = my_url)
    data = r.json()
    terms = data['drugMemberGroup']['drugMember']
    lst = []

    for n, term in enumerate(terms):
        rxcui = terms[n]['minConcept']['name']
        lst.append(rxcui)
    
    return lst

In [39]:

def get_ndc(rxcui_lst):
    '''Return ndc for rxcui'''

    for rxcui in rxcui_lst:
        url_base = 'https://rxnav.nlm.nih.gov/REST/rxcui/'
        my_url = url_base + rxcui + '/ndcs.json'

        r = requests.get(url = my_url)
        data = r.json() 
        
        terms = data['ndcGroup']['ndcList']
        ndc_lst = []

        if len(terms) > 0:
            terms2 = data['ndcGroup']['ndcList']['ndc']

            for term in terms2:
                ndc_lst.append(term)
    
    return ndc_lst


In [40]:
opiods = get_drug_name('N02A','ATC')
benzos = get_drug_name('D001569','DAILYMED')
dihyds = get_drug_name('D004095','DAILYMED')

In [ ]:
conn = sqlite3.connect('mimic.db')
cursor = conn.cursor()

### Creating tables

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Patients")
cursor.execute("DROP TABLE IF EXISTS Admissions")
cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
cursor.execute("DROP TABLE IF EXISTS Prescriptions")


sql ='''CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );'''

cursor.execute(sql)


sql ='''CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''

cursor.execute(sql)

sql ='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10) NOT NULL,
    short_title varchar(50) NOT NULL,
    long_title varchar(300) NOT NULL,
    PRIMARY KEY (icd9_code) 
  );'''
cursor.execute(sql)


sql ='''CREATE TABLE Diagnoses_icd (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
     seq_num int NOT NULL,
     icd9_code varchar(10) NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code)
  );'''
cursor.execute(sql)


 #insert code to create Prescriptions table
sql ='''CREATE TABLE Prescriptions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    startdate datetime NULL,
    enddate datetime NULL,
    drug_type varchar(50) NULL,
    drug varchar(300) NULL,
    drug_name_poe varchar(300) NULL,
    drug_name_generic varchar(300) NULL,
    formulary_drug_cd varchar(300) NULL,
    gsn int NULL,
    ndc real NULL,
    prod_strength varchar(300) NULL,
    dose_val_rx varchar(50) NULL,
    dose_unit_rx varchar(50) NULL,
    form_val_disp varchar(50) NULL,
    form_unit_disp varchar(50) NULL,
    route varchar(50) NULL,
    FOREIGN KEY (hadm_id) REFERENCES Diagnoses_icd(hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
cursor.execute(sql)



print("Table(s) created successfully........")
conn.commit()

Table(s) created successfully........


### Data Cleaning 

In [ ]:
patients = pd.read_csv('../../mimic-iii-clinical-database-1.4/PATIENTS.csv.gz', compression='gzip')

In [ ]:
admissions = pd.read_csv('../../mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz', compression='gzip')

In [ ]:
d_icd_diagnoses = pd.read_csv('../../mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz', compression='gzip')

In [ ]:
diagnoses_icd = pd.read_csv('../../mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz', compression='gzip')

In [ ]:
prescriptions = pd.read_csv('../../mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz', compression='gzip', low_memory=False)

In [ ]:
prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [53]:
def find_drug(drug):
    return prescriptions.DRUG[prescriptions.DRUG.str.contains(drug, case=False, regex=True)].value_counts().to_frame()

In [ ]:
def clean_drug(drug):
    return prescriptions.DRUG[prescriptions.DRUG.str.replace(drug, drug, case=False, regex=True)]

In [48]:
opiods

['naloxone / tilidine',
 'tilidine',
 'tramadol',
 'aspirin / codeine',
 'naloxone / oxycodone',
 'naltrexone / oxycodone',
 'buprenorphine',
 'butorphanol',
 'dezocine',
 'dihydrocodeine',
 'oliceridine',
 'celecoxib / tramadol',
 'dextromoramide',
 'hydromorphone',
 'acetaminophen / tramadol',
 'fentanyl',
 'ibuprofen / oxycodone',
 'meperidine',
 'meptazinol',
 'morphine',
 'codeine / ibuprofen',
 'nalbuphine',
 'opium',
 'oxycodone',
 'oxymorphone',
 'tapentadol',
 'papaveretum',
 'pentazocine',
 'phenazocine',
 'acetaminophen / codeine',
 'pirinitramide',
 'propoxyphene']

In [55]:
find_drug('propoxyphene')

,DRUG
Propoxyphene Nap.-Apap (N-100),493
Propoxyphene HCl,10
Propoxyphene,2


In [ ]:

find_drug('midazolam')

Midazolam           13652
Midazolam HCl        7902
NEO*PO*Midazolam        6
MIDAZOLAM               3
Name: DRUG, dtype: int64

In [ ]:
13652 + 7902 + 6 + 3

21563

In [ ]:
find_drug('lorazepam')

Lorazepam                            55352
NEO*PO*Lorazepam Oral Concentrate       20
Lorazepam 2mg syringe                   14
LORAZEPAM                                8
Lorazepam 2mg Syringe                    2
Lorazepam 2mg syringe                    1
Lorazepam 2 mg syringe                   1
Lorazepam 2mg syringa                    1
Lorazepam 2 mg syringe                   1
Name: DRUG, dtype: int64

In [ ]:
find_drug('diazepam')

Diazepam    10399
Name: DRUG, dtype: int64

In [ ]:
find_drug('propofol')

Propofol                34462
Propofol (Generic)       3715
PROPOFOL                    5
Propofol (Diprivan)         3
PROPOFOL (*GENERIC*)        2
Propofol Diprivan           1
Name: DRUG, dtype: int64

In [ ]:
find_drug('dexmedetomidine')

Dexmedetomidine        2258
Dexmedetomidine HCl     943
Dexmedetomidine Hcl      16
Name: DRUG, dtype: int64

In [ ]:
find_drug('fentanyl')

Fentanyl Citrate         31669
Fentanyl Patch            3233
NEO*IV*Fentanyl            424
Fentanyl PCA               348
Fentanyl                    48
FENTANYL                     9
Fentanyl                     1
fentaNYL                     1
fentaNYL citrate (PF)        1
Name: DRUG, dtype: int64

In [ ]:
find_drug('morphine')

Morphine Sulfate                               62134
Morphine Sulfate IR                             1754
Morphine SR (MS Contin)                         1328
Morphine Sulfate (Oral Soln.)                   1032
Morphine Sulfate (Concentrated Oral Soln)        829
Morphine Sulfate SR                              608
NEO*IV*Morphine Sulfate                           90
NEO*PO*Morphine                                   70
Morphine Sulfate                                  25
Morphine (PF)                                     25
Morphine                                          21
Morphine Infusion ? Comfort Care Guidelines       18
Morphine                                          18
Morphine PCA                                      17
Morphine Sulfate (Syringe)                        13
Morphine Sulfate PF                               13
Morphine Sulfate (2-4mg)                           8
Morphine Sulfate (Immediate Release)               7
Morphine Sulfate For Inhalation               

In [ ]:
find_drug('Hydromorphone')

HYDROmorphone (Dilaudid)                            37673
Hydromorphone                                       11898
HYDROmorphone                                        1399
HYDROmorphone P.F.                                    938
Hydromorphone P.F.                                    828
HYDROmorphone-HP                                      189
Hydromorphone-HP                                      185
Hydromorphone HCl                                      27
Hydromorphone-hp                                       25
Hydromorphone PCA                                      17
HYDROmorphone PCA                                       8
Hydromorphone Hcl                                       3
HYDROmorphone PCA (Dilaudid)                            3
hydromorphone                                           2
Hydromorphone (Rectal)                                  2
HYDROmorphone Solution                                  2
HYDROmorphone Infusion ? Comfort Care Guidelines        1
HYDROmorphone 

###  Loading Cleaned Data into Database

In [ ]:
patients.to_sql('Patients', conn, if_exists='replace', index = False)
conn.commit()

admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
conn.commit()

d_icd_diagnoses.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
conn.commit()

diagnoses_icd.to_sql('Diagnoses_icd', conn, if_exists='replace', index = False)
conn.commit()

prescriptions.to_sql('Prescriptions', conn, if_exists='replace', index = False)
conn.commit()